In [ ]:
#| default_exp camera_settings

Write in a raw cell 
---  
skip_exec: true  
---  
to avoid executing the full notebook by tools like nbdev_test or Github CI/CD.

In [ ]:
%env QT_LOGGING_RULES="*.debug=false;qt.qpa.*=false"

env: QT_LOGGING_RULES="*.debug=false;qt.qpa.*=false"


# Camera parameters

1. Camera connections
2. Camera parameters
3. Camera properties
4. Camera calibration


## 1. Camera connections

The camera can be connected through different interfaces, such as USB, Ethernet, or Wi-Fi, and to different OS (Windows, Linux, macOS). The OS has to be able to recognize the camera and provide access to it.  
In Linux, cameras are typically accessed through the Video4Linux2 (V4L2) API, which provides a standardized interface for video capture devices. The camera is usually represented as a device file (e.g., `/dev/video0`).
In Windows, cameras are accessed through the DirectShow API, and they are typically represented as devices in the Device Manager.

- Connection method (USB, Ethernet, Wi-Fi)
    + Limits the bandwidth. USB 2.0 has a maximum bandwidth of 480 Mbps, while USB 3.0 can provide up to 5 Gbps. Ethernet connections can offer even higher bandwidths, depending on the network infrastructure.
    + Affects latency. USB connections typically have lower latency compared to Ethernet or Wi-Fi connections, which can be crucial for real-time applications.
    + Others: mobility, power consumption, etc.
- OS compatibility
    + Linux: V4L2 API, device files (e.g., `/dev/video0`)
    + Windows: DirectShow API, Device Manager
    + macOS: AVFoundation framework, device files (e.g., `/dev/video0`)

- Bridges (WSL2 & USBIPD):

    + If the camera is connected to a virtualized environment like WSL2, it requires a bridge to "pass through" the physical hardware from the host (Windows).

    + USBIPD: This tool acts as a protocol encapsulator (USB-over-IP). It wraps physical USB data into network packets to transport them across the virtual network switch into the Linux kernel.

    + Performance Impact: Because it adds layers of encapsulation, bridges introduce micro-latency and jitter. This often breaks synchronization for high-bandwidth, uncompressed streams. (Problems transmitting YUYV raw format, but works with MJPG compressed format)

### On Windows

Powershell commands to install and use USBIPD:

``` PowerShell
winget install usbipd
usbipd list
usbipd bind --busid <busid>
usbipd attach --wsl --busid <busid>   (You need an opened WSL terminal to run this command)
```

### On Linux


Commands to list video devices:
```bash
ls /dev/video*
```

Commands to list USB devices:
```bash
lsusb
```

Commands to test the camera connection:
``` bash
v4l2-ctl -d /dev/video0 --list-formats-ext
```

Commands to test the camera with different formats:
``` bash
ffplay -f v4l2 -input_format yuyv422 -video_size 640x480 -i /dev/video0
```

``` bash
ffplay -f v4l2 -input_format mjpeg -video_size 1920x1080 -i /dev/video0
```

### With OpenCV

In [ ]:
#| export
import cv2
import time


Set the format that the camera should use to transmit the video stream. Raw images (YUYV) (bad performance with WSL2) or compressed images (MJPG) (good performance lower color fidelity).

In [ ]:
#| export
def set_cam_format_fourcc(cam, format='MJPG'):
    "Set Pixel or Compression format"
    # Options include 'MJPG', 'YUYV', 'RGB3', etc. Check your camera documentation for supported formats.
    # Check that the format is valid, otherwise return an error message
    valid_formats = ['MJPG', 'YUYV', 'RGB3', 'GRAY', 'YUV420P', 'NV12', 'NV21']
    if format not in valid_formats:
        raise ValueError(f"Invalid format '{format}'. Valid formats are: {valid_formats}")

    # Set the camera to output raw frames (uncompressed)
    cam.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*format))

def get_cam_format_fourcc(cam):
    "Get the current Pixel or Compression format"
    codec_int = int(cam.get(cv2.CAP_PROP_FOURCC))
    codec_str = "".join([chr((codec_int >> 8 * i) & 0xFF) for i in range(4)])
    return codec_str


In [ ]:
camera_id = 0
cap = cv2.VideoCapture(camera_id)
set_cam_format_fourcc(cap, 'YUYV') # Critical for WSL timeouts
print(f"Camera format: {get_cam_format_fourcc(cap)}")

set_cam_format_fourcc(cap, 'MJPG') # Critical for WSL timeouts
print(f"Camera format: {get_cam_format_fourcc(cap)}")

try:
    set_cam_format_fourcc(cap, 'INVALID') # Invalid format, should raise an error
except ValueError as e:
    print(e)

cap.release()


Camera format: YUYV
Camera format: MJPG
Invalid format 'INVALID'. Valid formats are: ['MJPG', 'YUYV', 'RGB3', 'GRAY', 'YUV420P', 'NV12', 'NV21']


Set the resolution of the image stream. Higher resolutions require more bandwidth and processing power, which can lead to increased latency and reduced frame rates.

In [ ]:
#| export
def set_cam_resolution(cam, width=640, height=480):
    "Set the resolution of the camera feed"
    cam.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

def get_cam_resolution(cam):
    "Get the current resolution of the camera feed"
    width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return width, height

Test that the camera can be opened and read frames from it:


In [ ]:
#| export
def test_camera_feed(camera_id=0, format='MJPG', width=640, height=480):
    cap = cv2.VideoCapture(camera_id)
    set_cam_format_fourcc(cap, format)
    set_cam_resolution(cap, width, height)

    if not cap.isOpened():
        print("Cannot open camera")
        return

    print(f"Camera format: {get_cam_format_fourcc(cap)}")
    print(f"Camera resolution: {get_cam_resolution(cap)}")

    print("Press 'q' to quit")


    prev_frame_time = 0
    new_frame_time = 0

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            # If it fails, print the specific error
            print("Failed to grab frame")
            break

        # Calculate FPS
        new_frame_time = time.time()
        fps = 1 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        
        # Format and draw the FPS on the frame
        fps_text = f"FPS: {int(fps)}"
        cv2.putText(frame, fps_text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)    
        
        cv2.imshow('Live Camera', frame)
        
        # Use Esc or 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF in [27, ord('q')]:  # Esc key or 'q'
            print("Exiting...")
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
test_camera_feed(width=1280, height=720)

Camera format: MJPG
Camera resolution: (1280, 720)
Press 'q' to quit
Exiting...


# More on OpenCV

We can provide an index to point to connected cameras (e.g., USB) or a URL for cameras connected wirelessly to the network

In [ ]:

# camera_id = 0 # or an url for an IP camera "192.168.1.100:8080/video"
camera_id = "http://192.168.55.113:8080/video"

test_camera_feed(camera_id=camera_id, format='MJPG', width=640, height=480)


Cannot open camera


[tcp @ 0x1f76da40] Connection to tcp://192.168.55.113:8080 failed: Connection refused


Main loop to show the camera feed.

In [ ]:
print("Press 'q' to quit")

camera_id = 0 # or an url for an IP camera "
cap = cv2.VideoCapture(camera_id)

# Let's avoid timeout errors in WSL, since the camera may want to transmit uncompressed frames by default 
set_cam_format_fourcc(cap, 'MJPG')

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Display the frame
    cv2.imshow('Live Camera', frame)
    
    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Press 'q' to quit


Notice that with `cap = cv2.VideoCapture(camera_id)` we instantiate the capture object, and at the end we need to release it with `cap.release()`. This is important to free up the camera resource for other applications or future runs of the script.

Autofocus and autoexposure can be disabled to have more control over the camera parameters

In [ ]:
# 1. Desactivate autofocus to avoid focus hunting and latency issues
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)

False

Manually set the focus


In [ ]:
cap.set(cv2.CAP_PROP_FOCUS, 40)

False

Interactive example:

In [ ]:
# 2. Create the window before the slider
cap = cv2.VideoCapture(0)

# Functions to set and get camera format and resolution
set_cam_format_fourcc(cap, 'MJPG') # Critical for WSL timeouts
set_cam_resolution(cap, 640, 480)

# Create a callback function for the slider
def on_change(val):
    cap.set(cv2.CAP_PROP_FOCUS, val)

# 2. Create the window before the slider
window_name = 'Camera Settings'
cv2.namedWindow(window_name)

# Create a slider for focus (0-1023)
cv2.createTrackbar('Focus', window_name, 439, 1023, on_change)

# Disable Auto Focus first
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)


while True:
    ret, frame = cap.read()
    if not ret: break
    
    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()